# import packages

In [9]:
library(DESeq2)
library(data.table)
library(dplyr)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(clusterProfiler)
library(AnnotationDbi)
library(org.Hs.eg.db)
library(ggplot2)
library(gridExtra)
library(IRdisplay)
library(ChIPpeakAnno)
library(IRdisplay)
library(ggpubr)
library(DiffBind)
library(kableExtra)
library(tidyverse)
library(readxl)
library(umap)
library(ggrepel)
library(msigdbr)
library(fgsea)
library(ggvenn)
library(ComplexHeatmap)
library(ggVennDiagram)
library(venneuler)
library(sva)
library(DT)
library(tools)
library(tidyverse)
library(R.utils)

library(edgeR)


In [10]:
source('~/script/utils.r')
wk_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/'
setwd(wk_dir)
data_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Blueprint_EGA_Data_EGAD00001000745_Transcriptional_Diversity_Human_Blood_progenitors_Chen_etal_Science_2014/' 

Sys.setenv(PATH='/hpcf/authorized_apps/rhel8_apps/mamba/install/mambaforge/sbin:/hpcf/authorized_apps/rhel8_apps/mamba/install/mambaforge/bin:/opt/TurboVNC/bin:/cm/local/apps/environment-modules/4.5.3/bin:/hpcf/lsf/lsf_prod/10.1/linux3.10-glibc2.17-x86_64/etc:/hpcf/lsf/lsf_prod/10.1/linux3.10-glibc2.17-x86_64/bin:/usr/lpp/mmfs/bin:/usr/lpp/mmfs/lib:/home/whu78/conda3/envs/te/bin:/condabin:/home/whu78/conda3/bin:/usr/local/sbin:/usr/sbin:/sbin:/usr/sbin:/cm/local/apps/environment-modules/4.5.3/bin:/home/whu78/.local/bin:/home/whu78/bin:/usr/sbin:/sbin:/bin:/usr/bin:/etc:/usr/ucb:/usr/local/bin:/usr/local/local_dfs/bin:/usr/bin/X11:/usr/local/sas:/usr/sbin:/sbin:/bin:/usr/bin:/etc:/usr/ucb:/usr/local/bin:/usr/local/local_dfs/bin:/usr/bin/X11:/usr/local/sas:/home/whu78/edirect')


# design table

In [ ]:
# ln -s /research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages/ /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac


In [39]:
flist = dir('fastq_atac/')
flist = flist[grep('fastq.gz', flist)]
dsn_kl = data.table(fname = flist)
dsn_kl[, srr_id := sub('.*(SRR\\d+).*', '\\1', fname)]
dsn_kl[, sample_name := str_extract(fname, '(.*)_', group = 1)]
dsn_kl[, sample_name := sub('_SRR.*', '', sample_name)]
dsn_kl[, fastq := paste0(wk_dir, 'fastq_atac/', fname)]

dsn_kl[, r1r2 := 'R1'] 
dsn_kl[grep('_R2.fastq.gz', fname), r1r2 := 'R2']
dsn_kl = dcast(sample_name + srr_id ~ r1r2, data = dsn_kl, value.var = 'fastq')


In [40]:
head(dsn_kl)

sample_name,srr_id,R1,R2
<chr>,<chr>,<chr>,<chr>
BFUE_ATAC_1,SRR8723726,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac/BFUE_ATAC_1_SRR8723726.fastq.gz,NA
BFUE_ATAC_2,SRR8723727,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac/BFUE_ATAC_2_SRR8723727.fastq.gz,NA
B_1,SRR2920492,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac/B_1_SRR2920492_R1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac/B_1_SRR2920492_R2.fastq.gz
B_2,SRR2920513,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac/B_2_SRR2920513_R1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac/B_2_SRR2920513_R2.fastq.gz
CD4_T_1,SRR2920493,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac/CD4_T_1_SRR2920493_R1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac/CD4_T_1_SRR2920493_R2.fastq.gz
CD4_T_2,SRR2920496,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac/CD4_T_2_SRR2920496_R1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/fastq_atac/CD4_T_2_SRR2920496_R2.fastq.gz


# prepare design table for GSE74912

In [41]:
dsn_gse74912 = fread('/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/gse74912_flist')
dsn_gse74912[, srx_id := {system(paste0('esearch -db sra -query "', gsm_id, '" | efetch -format docsum | accn-at-a-time | grep -i srx | tr [:lower:] [:upper:] | tr "\n" "," '), intern = T)}, 
      by = 1:nrow(dsn_gse74912)]
dsn_gse74912[, srr_id := {system(paste0('esearch -db sra -query "', gsm_id, '" | efetch -format docsum | accn-at-a-time | grep -i srr | tr [:lower:] [:upper:] | tr "\n" "," '), intern = T)}, 
      by = 1:nrow(dsn_gse74912)]
      

In [ ]:
# test if there is multiple ids
dsn_gse74912[, nn := length(unlist(strsplit(srx_id, ','))), by = 1:nrow(dsn_gse74912)]
table(dsn_gse74912$nn)
dsn_gse74912[, nn := length(unlist(strsplit(srr_id, ','))), by = 1:nrow(dsn_gse74912)]
table(dsn_gse74912$nn)

In [ ]:
dsn_gse74912[, srx_id := sub(',', '', srx_id)]
dsn_gse74912[, srr_id := sub(',', '', srr_id)]

In [ ]:
head(dsn_gse74912)

In [ ]:
dsn_gse74912$batch = 'gse74912'


In [42]:
#fwrite(dsn_gse74912, file = 'dsn_gse74912.tsv', sep = '\t')
dsn_gse74912 = fread('dsn_gse74912.tsv')


In [56]:
head(dsn_gse74912)

gsm_id,sample_name,srx_id,srr_id,nn,batch
<chr>,<chr>,<chr>,<chr>,<int>,<chr>
GSM1937376,4983-1A,SRX1427789,SRR2920466,1,gse74912
GSM1937377,4983-2A,SRX1427790,SRR2920467,1,gse74912
GSM1937378,4983-3A,SRX1427791,SRR2920468,1,gse74912
GSM1937379,4983-4A,SRX1427792,SRR2920469,1,gse74912
GSM1937380,4983-4B,SRX1427793,SRR2920470,1,gse74912
GSM1937381,4983-5A,SRX1427794,SRR2920471,1,gse74912


# prepare design table fro GSE128266


In [127]:
dsn_gse128266 = fread('/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/gse128266_flist')
dsn_gse128266[, srx_id := {system(paste0('esearch -db sra -query "', gsm_id, '" | efetch -format docsum | accn-at-a-time | grep -i srx | tr [:lower:] [:upper:] | tr "\n" "," '), intern = T)}, 
      by = 1:nrow(dsn_gse128266)]
dsn_gse128266[, srr_id := {system(paste0('esearch -db sra -query "', gsm_id, '" | efetch -format docsum | accn-at-a-time | grep -i srr | tr [:lower:] [:upper:] | tr "\n" "," '), intern = T)}, 
      by = 1:nrow(dsn_gse128266)]
      

In [129]:
# test if there is multiple ids
dsn_gse128266[, nn := length(unlist(strsplit(srx_id, ','))), by = 1:nrow(dsn_gse128266)]
table(dsn_gse128266$nn)
dsn_gse128266[, nn := length(unlist(strsplit(srr_id, ','))), by = 1:nrow(dsn_gse128266)]
table(dsn_gse128266$nn)



 1 
16 


 1 
16 

In [131]:
dsn_gse128266[, srx_id := sub(',', '', srx_id)]
dsn_gse128266[, srr_id := sub(',', '', srr_id)]


In [140]:
dsn_gse128266$batch = 'gse128266'

In [43]:
#fwrite(dsn_gse128266, file = 'dsn_gse128266.tsv', sep = '\t')
dsn_gse128266 = fread('dsn_gse128266.tsv')

In [44]:
head(dsn_gse128266)

gsm_id,sample_name,srx_id,srr_id,nn,batch
<chr>,<chr>,<chr>,<chr>,<int>,<chr>
GSM3669542,HSPC_ATAC_1,SRX5516951,SRR8723724,1,gse128266
GSM3669543,HSPC_ATAC_2,SRX5516952,SRR8723725,1,gse128266
GSM3669544,BFUE_ATAC_1,SRX5516953,SRR8723726,1,gse128266
GSM3669545,BFUE_ATAC_2,SRX5516954,SRR8723727,1,gse128266
GSM3669546,CFUE_ATAC_1,SRX5516955,SRR8723728,1,gse128266
GSM3669547,CFUE_ATAC_2,SRX5516956,SRR8723729,1,gse128266


# merge design table

In [45]:
dsn_gse = rbindlist(list(dsn_gse74912, dsn_gse128266))
head(dsn_gse)


gsm_id,sample_name,srx_id,srr_id,nn,batch
<chr>,<chr>,<chr>,<chr>,<int>,<chr>
GSM1937376,4983-1A,SRX1427789,SRR2920466,1,gse74912
GSM1937377,4983-2A,SRX1427790,SRR2920467,1,gse74912
GSM1937378,4983-3A,SRX1427791,SRR2920468,1,gse74912
GSM1937379,4983-4A,SRX1427792,SRR2920469,1,gse74912
GSM1937380,4983-4B,SRX1427793,SRR2920470,1,gse74912
GSM1937381,4983-5A,SRX1427794,SRR2920471,1,gse74912


In [46]:
nrow(dsn_gse)
nrow(dsn_kl)
intersect(dsn_gse$srr_id, dsn_kl$srr_id)
setdiff(dsn$srr_id, dsn_gse$srr_id)

[1] 146

[1] 46

[1] "SRR2920492" "SRR2920493" "SRR2920494" "SRR2920495" "SRR2920496"
 [6] "SRR2920497" "SRR2920498" "SRR2920499" "SRR2920500" "SRR2920501"
[11] "SRR2920502" "SRR2920505" "SRR2920506" "SRR2920507" "SRR2920508"
[16] "SRR2920509" "SRR2920510" "SRR2920511" "SRR2920513" "SRR2920523"
[21] "SRR8723724" "SRR8723725" "SRR8723726" "SRR8723727" "SRR8723728"
[26] "SRR8723729" "SRR8723730" "SRR8723731" "SRR8723732" "SRR8723733"
[31] "SRR8723734" "SRR8723735" "SRR8723736" "SRR8723737" "SRR8723738"
[36] "SRR8723739"

[1] "SRR2999324" "SRR2999325" "SRR5210376" "SRR7650849" "SRR8247848"
 [6] "SRR8247849" "SRR8247850" "SRR8247851" "SRR8247852" "SRR8247853"

In [47]:
dsn_merged = merge(dsn_kl, dsn_gse, by = 'srr_id', all = T)
dsn_merged[is.na(sample_name.y), sample_name.y := sample_name.x]
dsn_merged[, sample_name := sample_name.y]
#dsn_merged[, sample_name.x := NULL ]
#dsn_merged[, sample_name.y := NULL ]
head(dsn_merged)


srr_id,sample_name.x,R1,R2,gsm_id,sample_name.y,srx_id,nn,batch,sample_name
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>
SRR2920466,NA,NA,NA,GSM1937376,4983-1A,SRX1427789,1,gse74912,4983-1A
SRR2920467,NA,NA,NA,GSM1937377,4983-2A,SRX1427790,1,gse74912,4983-2A
SRR2920468,NA,NA,NA,GSM1937378,4983-3A,SRX1427791,1,gse74912,4983-3A
SRR2920469,NA,NA,NA,GSM1937379,4983-4A,SRX1427792,1,gse74912,4983-4A
SRR2920470,NA,NA,NA,GSM1937380,4983-4B,SRX1427793,1,gse74912,4983-4B
SRR2920471,NA,NA,NA,GSM1937381,4983-5A,SRX1427794,1,gse74912,4983-5A


In [49]:
dsn_merged[!is.na( sample_name.x ) & is.na(sample_name.y), ]

srr_id,sample_name.x,R1,R2,gsm_id,sample_name.y,srx_id,nn,batch,sample_name
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>


# download other dataset

In [50]:
dsn = copy(dsn_merged)


In [51]:
fastq_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/'
dsn[, prefetch_cmd := paste0('prefetch ', srr_id, ' -O ', fastq_dir, ' --max-size 40g')]
# if single end seq
dsn[, fastqdump_cmd := paste0('fasterq-dump ', srr_id,  ' --outdir ', fastq_dir, srr_id, '/')]
dsn[, dump_r := paste0(fastq_dir, srr_id, '/', srr_id, '.fastq')]
# if double end seq
dsn[, dump_r1 := paste0(fastq_dir, srr_id, '/', srr_id, '_1.fastq')] 
dsn[, dump_r2 := paste0(fastq_dir, srr_id, '/', srr_id, '_2.fastq')]
dsn[, gz_r_cmd := paste0(' gzip ', dump_r)]
dsn[, gz_r1_cmd := paste0(' gzip ', dump_r1)]
dsn[, gz_r2_cmd := paste0(' gzip ', dump_r2)]


In [ ]:
# script for downloading SRA => fastq => gzip
write(dsn[!file.exists(sra_file) & !file.exists(R1), prefetch_cmd], file = paste0(fastq_dir, 'run_batch2_prefetch.sh'))
write(dsn[file.exists(sra_file) & !(file.exists(dump_r) | file.exists(dump_r1)), fastqdump_cmd], file = paste0(fastq_dir, 'run_batch2_fastqdump_cmd.sh'))


In [537]:
# compress fastq files
dsn[, dump_r_gz := paste0(fastq_dir, srr_id, '/', srr_id, '.fastq.gz')]
# if double end seq
dsn[, dump_r1_gz := paste0(fastq_dir, srr_id, '/', srr_id, '_1.fastq.gz')]
dsn[, dump_r2_gz := paste0(fastq_dir, srr_id, '/', srr_id, '_2.fastq.gz')]

write(dsn[file.exists(dump_r)  & !file.exists(dump_r_gz), gz_r_cmd], file = paste0(wk_dir, 'run_', PP, '_gz_r_cmd.sh'))
write(dsn[file.exists(dump_r1) & !file.exists(dump_r1_gz), gz_r1_cmd], file = paste0(wk_dir, 'run_', PP, '_gz_r1_cmd.sh'))
write(dsn[file.exists(dump_r2) & !file.exists(dump_r2_gz), gz_r2_cmd], file = paste0(wk_dir, 'run_', PP, '_gz_r2_cmd.sh'))


In [549]:
# get the real file path, not link, easier for file test
dsn[, RR1 := Sys.readlink(R1)] 
dsn[, RR2 := Sys.readlink(R2)] 

In [542]:
sum(file.exists(dsn$dump_r1_gz) & !file.exists(dsn$RR1))
sum(!file.exists(dsn$dump_r1_gz) & file.exists(dsn$RR1))


[1] 110

[1] 46

In [485]:
dsn[, sra_file := paste0(fastq_dir, srr_id, '/', srr_id, '.sra')]
file.exists(dsn[!file.exists(dsn$R1), sra_file])


[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE

In [58]:
dsn[, sra_file := paste0(fastq_dir, srr_id, '/', srr_id, '.sra')]
file.exists(dsn[!file.exists(dsn$R1), sra_file])


[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE

In [59]:
file.exists(dsn$sra_file)

[1]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [13]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [25]  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [37] FALSE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
 [49]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE
 [61]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [73]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [85]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [97]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[109]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[121]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE
[133] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE  TRUE
[145] FALSE  TRUE FALSE  TRUE FALSE  TRUE FALSE  TRUE FALSE  TRUE FALSE  TRUE

In [65]:
# this should be 0
nrow(dsn[file.exists(dump_r) & file.exists(dump_r1), ])


[1] 0

In [66]:
dsn[!file.exists(RR1) & file.exists(dump_r1_gz), RR1 := dump_r1_gz]
dsn[!file.exists(RR2) & file.exists(dump_r2_gz), RR2 := dump_r2_gz]

# there is no such situation
# dsn_merged[!file.exists(R1) & file.exists(dump_ri_gz), RR1 := dump_r_gz]


In [547]:
dim(dsn[!file.exists(RR1), ])

[1] 110  32

In [552]:
all(file.exists(dsn$RR1))
all(file.exists(dsn$RR2))
     

[1] TRUE

[1] FALSE

# export design table

In [1]:
#fwrite(dsn, file = 'data/dsn_te_atac_merged.xls', sep = '\t')
# manual check the files
#dsn = fread('data/dsn_te_atac_merged_bak.txt') 
dsn = fread('data/dsn_te_atac_merged_v2.xls') 


ERROR: Error in fread("data/dsn_te_atac_merged_v2.xls"): could not find function "fread"


In [2]:
file.exists(dsn$RR2)

ERROR: Error: object 'dsn' not found


In [75]:
colnames(dsn)

[1] "srr_id"             "sample_name"        "sample_name_manual"
 [4] "R1"                 "R2"                 "gsm_id"            
 [7] "srx_id"             "nn"                 "batch"             
[10] "sra_file"           "prefetch_cmd"       "fastqdump_cmd"     
[13] "dump_r"             "dump_r1"            "dump_r2"           
[16] "gz_r_cmd"           "gz_r1_cmd"          "gz_r2_cmd"         
[19] "RR1"                "RR2"                "sample_dir"        
[22] "fname_pre"          "fastq_r1_trimmed"   "fastq_r2_trimmed"  
[25] "cutadapt_jobname"   "cutadapt_oo"        "cutadapt_eo"       
[28] "cutadapt_cmd"       "dump_r_gz"          "dump_r1_gz"        
[31] "dump_r2_gz"

In [72]:
fastq_dir

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/"

In [82]:
base_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/'
fwrite(dsn[, .(sample_name, sample_name_manual, gsm_id, batch, RR1, RR2)], file = paste0(base_dir, 'share_table_hema_atac.tsv'), sep = '\t')

In [81]:
head(dsn[, .(sample_name, sample_name_manual, gsm_id, batch, RR1, RR2)])

sample_name,sample_name_manual,gsm_id,batch,RR1,RR2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
4983-1A,D4983_HSC_1,GSM1937376,gse74912,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920466/SRR2920466_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920466/SRR2920466_2.fastq.gz
4983-2A,D4983_HSC_1,GSM1937377,gse74912,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920467/SRR2920467_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920467/SRR2920467_2.fastq.gz
4983-3A,D4983_HSC_1,GSM1937378,gse74912,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920468/SRR2920468_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920468/SRR2920468_2.fastq.gz
4983-4A,D4983_HSC_1,GSM1937379,gse74912,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920469/SRR2920469_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920469/SRR2920469_2.fastq.gz
4983-4B,D4983_HSC_1,GSM1937380,gse74912,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920470/SRR2920470_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920470/SRR2920470_2.fastq.gz
4983-5A,D4983_HSC_1,GSM1937381,gse74912,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920471/SRR2920471_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920471/SRR2920471_2.fastq.gz


In [32]:
colnames(dsn)


[1] "srr_id"             "sample_name"        "sample_name_manual"
 [4] "R1"                 "R2"                 "gsm_id"            
 [7] "srx_id"             "nn"                 "batch"             
[10] "sra_file"           "prefetch_cmd"       "fastqdump_cmd"     
[13] "dump_r"             "dump_r1"            "dump_r2"           
[16] "gz_r_cmd"           "gz_r1_cmd"          "gz_r2_cmd"         
[19] "RR1"                "RR2"                "sample_dir"        
[22] "fname_pre"          "fastq_r1_trimmed"   "fastq_r2_trimmed"  
[25] "cutadapt_jobname"   "cutadapt_oo"        "cutadapt_eo"       
[28] "cutadapt_cmd"       "dump_r_gz"          "dump_r1_gz"        
[31] "dump_r2_gz"

In [85]:
dsn[, cell_type := unlist(strsplit(sample_name_manual, '_'))[2], by = 1:nrow(dsn)]

In [86]:
table(dsn$cell_type)


      B    BFUE   blast  CBCD34    CD34     CD4     CD8    CFUE     CLP     CMP 
      4       2      23       1       2       5       5       2       5       2 
     DC      EB   Ebaso     GMP     HSC    HSPC     LSC     MEP    Mono     MPP 
      1       8       4       1      38       2       8       1       1       2 
    Neu      NK  orthoE    pHSC   polyE preproB    ProB    proE 
      2       6       2      19       2       3       3       2 

In [83]:
table(dsn$batch)


          gse128266  gse74912 
       10        16       130 

In [36]:
head(dsn[!file.exists(dump_r_gz) & !file.exists(dump_r1_gz), .(sample_name_manual, dump_r_gz, dump_r1_gz, dump_r2_gz)])


sample_name_manual,dump_r_gz,dump_r1_gz,dump_r2_gz
<chr>,<chr>,<chr>,<chr>
D1022_B_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920492/SRR2920492.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920492/SRR2920492_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920492/SRR2920492_2.fastq.gz
D1022_CD4_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920493/SRR2920493.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920493/SRR2920493_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920493/SRR2920493_2.fastq.gz
D1022_CD8_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920494/SRR2920494.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920494/SRR2920494_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920494/SRR2920494_2.fastq.gz
D1022_NK_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920495/SRR2920495.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920495/SRR2920495_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920495/SRR2920495_2.fastq.gz
D2596_CD4_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920496/SRR2920496.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920496/SRR2920496_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920496/SRR2920496_2.fastq.gz
D2596_CD8_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920497/SRR2920497.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920497/SRR2920497_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920497/SRR2920497_2.fastq.gz


In [23]:
head(dsn[, .(sample_name, dump_r_gz, dump_r1_gz, dump_r2_gz)]) 

sample_name,dump_r_gz,dump_r1_gz,dump_r2_gz
<chr>,<chr>,<chr>,<chr>
4983-1A,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920466/SRR2920466.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920466/SRR2920466_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920466/SRR2920466_2.fastq.gz
4983-2A,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920467/SRR2920467.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920467/SRR2920467_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920467/SRR2920467_2.fastq.gz
4983-3A,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920468/SRR2920468.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920468/SRR2920468_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920468/SRR2920468_2.fastq.gz
4983-4A,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920469/SRR2920469.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920469/SRR2920469_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920469/SRR2920469_2.fastq.gz
4983-4B,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920470/SRR2920470.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920470/SRR2920470_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920470/SRR2920470_2.fastq.gz
4983-5A,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920471/SRR2920471.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920471/SRR2920471_1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920471/SRR2920471_2.fastq.gz


# create sample dir

In [29]:
wk_dir

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/"

In [560]:
run_dir = paste0(wk_dir, 'data/run_atac/')
dsn[, sample_dir := paste0(run_dir, sample_name, '/')]
dsn[, fname_pre := paste0(run_dir, sample_name, '_', species)]
PP = 'atac'


In [ ]:
# run_dir = paste0(wk_dir, 'data/run_atac/')
dir.create(run_dir, showWarnings = F)
dsn[, {dir.create(sample_dir, showWarnings = F)}, by = 1:nrow(dsn)]


In [562]:
all(dir.exists(dsn$sample_dir))

[1] TRUE

# merge_multi_lanes


In [599]:
library(R.utils)

In [629]:
se_sample = c('D1_HSPC_1',  'D2_HSPC_2',  'D3_BFUE_1',  'D4_BFUE_1',  'D5_CFUE_1',  'D6_CFUE_1',  'D7_proE_1',  'D8_proE_1',  'D9_Ebaso_1', 
  'D10_Ebaso_1',  'D11_Ebaso_1',  'D12_Ebaso_1',  'D13_polyE_1',  'D14_polyE_1',  'D15_orthoE_1',  'D16_orthoE_1') 
pe_sample = setdiff(dsn$sample_name, se_sample)

In [ ]:
dsn[, nn := nrow(.SD), by = 'sample_name_manual']
dsn[nn == 1, RR1_sample := RR1]
dsn[nn == 1, RR2_sample := RR2]
dsn[, sample_dir2 := paste0(wk_dir, 'data/run_atac_merged/', sample_name_manual, '/')]
dsn[, {dir.create(sample_dir2, showWarnings = F, recursive = T)}, by = 1:nrow(dsn)] 
dsn[, RR1_sample := paste0(sample_dir2, sample_name_manual, '_R1.fastq.gz')]
dsn[, RR2_sample := paste0(sample_dir2, sample_name_manual, '_R2.fastq.gz')]
dsn[, {if(.SD$nn == 1){createLink(link = RR1_sample, target = RR1)}}, by = 1:nrow(dsn)] 
dsn[, {if(.SD$nn == 1){createLink(link = RR2_sample, target = RR2)}}, by = 1:nrow(dsn)] 


In [621]:
dsn[, merge_jobname := paste0('cuta_', sample_name)]
dsn[, merge_oo := paste0(sample_dir, sample_name, '_', merge_jobname, '_out')]
dsn[, merge_eo := paste0(sample_dir, sample_name, '_', merge_jobname, '_err')]
dsn[, merge_cmd := paste0('bsub -J ', merge_jobname, ' -n 10 -o ', merge_oo, ' -e ', merge_eo)]
dsn[,  merge_cmd := paste0(merge_cmd, ' " cat ', paste(.SD$RR1, collapse = ' '), ' > ', RR1_sample, '"')]

write(dsn[nn > 1, ][!duplicated(sample_name_manual), merge_cmd], file = paste0(wk_dir, 'run_', PP, '_r1_merge.sh'))


In [622]:
dsn[, merge_jobname := paste0('cuta_', sample_name)]
dsn[, merge_oo := paste0(sample_dir, sample_name, '_', merge_jobname, '_out')]
dsn[, merge_eo := paste0(sample_dir, sample_name, '_', merge_jobname, '_err')]
dsn[, merge_cmd := paste0('bsub -J ', merge_jobname, ' -n 10 -o ', merge_oo, ' -e ', merge_eo)]
dsn[,  merge_cmd := paste0(merge_cmd, ' " cat ', paste(.SD$RR2, collapse = ' '), ' > ', RR2_sample, '"')]

write(dsn[nn > 1, ][!duplicated(sample_name_manual), merge_cmd], file = paste0(wk_dir, 'run_', PP, '_r2_merge.sh'))


In [626]:
# checkpoint 
#dsn_ori = copy(dsn)


In [627]:
# checkpoint for design table
dsn = dsn[!duplicated(sample_name_manual), ]
dsn[, sample_name := sample_name_manual] 
dsn[, sample_dir := sample_dir2] 


In [658]:
head(dsn$sample_dir)

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/"   
[2] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D6792_HSC_1/"   
[3] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D7291_CD34_1/"  
[4] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D7292_CD34_2/"  
[5] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D1020_CBCD34_1/"
[6] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D1022_B_1/"

# sample summary

In [646]:
dim(dsn)
dsn[, donor := unlist(strsplit(sample_name_manual, '_'))[1], by = 1:nrow(dsn)]
dsn[, cell_type := unlist(strsplit(sample_name_manual, '_'))[2], by = 1:nrow(dsn)]
dsn[, replicate := unlist(strsplit(sample_name_manual, '_'))[3], by = 1:nrow(dsn)]
dsn[, donor_type := 'ctrl']
dsn[grep('SU', donor), donor_type := 'AML']


[1] 96 43

In [647]:
table(dsn$donor_type)


 AML ctrl 
  38   58 

In [649]:
library(knitr)
library(kableExtra)
library(IRdisplay)


In [651]:
dsn[, .(sample_name_manual, donor_type, cell_type, replicate, batch)] %>%  
    kable("html") %>%
    as.character() %>%
    display_html()

sample_name_manual,donor_type,cell_type,replicate,batch
D4983_HSC_1,ctrl,HSC,1,gse74912
D6792_HSC_1,ctrl,HSC,1,gse74912
D7291_CD34_1,ctrl,CD34,1,gse74912
D7292_CD34_2,ctrl,CD34,2,gse74912
D1020_CBCD34_1,ctrl,CBCD34,1,gse74912
D1022_B_1,ctrl,B,1,gse74912
D1022_CD4_1,ctrl,CD4,1,gse74912
D1022_CD8_1,ctrl,CD8,1,gse74912
D1022_NK_1,ctrl,NK,1,gse74912
D2596_CD4_1,ctrl,CD4,1,gse74912


# cutadapt

In [625]:
run_dir
wk_dir

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac/"

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/"

In [638]:
# module load cutadapt/4.4

dsn[, fastq_r1_trimmed := paste0(sample_dir, sample_name, '_R1_trimmed.fa.gz')]
dsn[, fastq_r2_trimmed := paste0(sample_dir, sample_name, '_R2_trimmed.fa.gz')]
dsn[, cutadapt_jobname := paste0('cuta_', sample_name)]
dsn[, cutadapt_oo := paste0(sample_dir, sample_name, '_', cutadapt_jobname, '_out')]
dsn[, cutadapt_eo := paste0(sample_dir, sample_name, '_', cutadapt_jobname, '_err')]
dsn[, cutadapt_cmd := paste0('bsub -J ', cutadapt_jobname, ' -n 10 -o ', cutadapt_oo, ' -e ', cutadapt_eo)]
dsn[file.exists(RR2),  cutadapt_cmd := paste0(cutadapt_cmd, ' " cutadapt -o ', fastq_r1_trimmed, ' -p ', fastq_r2_trimmed, ' ', RR1, ' ', RR2, '"')]
dsn[!file.exists(RR2), cutadapt_cmd := paste0(cutadapt_cmd, ' " cutadapt -o ', fastq_r1_trimmed, ' ', RR1, '"')]
head(dsn[!file.exists(fastq_r1_trimmed) | file.size(fastq_r1_trimmed) < 500000, cutadapt_cmd], 1)
#write(dsn[count_file == 'No', cutadapt_cmd], file = paste0(base_dir, 'run_cutadapt.sh'))
write(dsn[!file.exists(fastq_r1_trimmed) | file.size(fastq_r1_trimmed) < 500000, cutadapt_cmd], file = paste0(wk_dir, 'run_', PP, '_cutadapt.sh'))


[1] "bsub -J cuta_D4983_HSC_1 -n 10 -o /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/D4983_HSC_1_cuta_D4983_HSC_1_out -e /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/D4983_HSC_1_cuta_D4983_HSC_1_err \" cutadapt -o /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/D4983_HSC_1_R1_trimmed.fa.gz -p /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/D4983_HSC_1_R2_trimmed.fa.gz /research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920466/SRR2920466_1.fastq.gz /research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch2/SRR2920466/SRR2920466_2.fastq.gz\""

In [641]:
dim(dsn)

[1] 96 40

In [653]:
file.size(dsn$fastq_r1_trimmed) 
file.size(dsn$fastq_r2_trimmed) 

[1]  295593895  387495564  564089177  599808635 1444371206 1324033923
 [7] 1769659901  816993928  754253628 3240031007 3208001779 1362454072
[13]  687213030 3231620075 1163059712 1168546873 1482469876 2638620433
[19] 3215339702 2513778298 1482895738 2579650723 2851321796 3009401534
[25] 3124111086 3127999946  999091216  550500661 2039111636  852691984
[31]  647975955 1901970408  320319616 1912233166 1353773482 1003910404
[37]  858084729  862248855 1115321752  867927013 1605396340 1373812190
[43] 1972551172 1913393025  999372154 1793644691  840201960 1311906716
[49] 1291690535 2257393716 2365907190 1163893450  865509989 1407609864
[55]  690809419 1663299208 2125135840 1430981768  985428413  971573121
[61] 1351674749 1169967692 2812698500 1303430077 3010608329 3080283130
[67] 1612948906 2166413415 1760633771 2028292471 2651364464 2777946013
[73] 2379597647 1010928649 1738950298 1286165397  748658691 1589411261
[79]  992435311 1121564084 2834463974 2845166483 2296422606 3407576443
[85] 4875398209 1512167418 1924731322 3433137994 2150353612 3145386217
[91] 2260687744 3260908030 2001040851 3402767230 1979298131 3507795052

[1]  302296866  393655827  673385362  714327616 1448836874 1354319864
 [7] 1842118627  846706479  756807115 3210796918 3167772852 1366611079
[13]  687854501 3239336537 1177361237 1179404433 1497075973 2662799217
[19] 3171322645 2510843396 1484856755 2573109647 2845627014 2986183097
[25] 3110763432 3123757515 1003128801  552644406 2021080352  841685582
[31]  639737607 1847101199  319432382 1924453809 1416219923 1038192330
[37]  884676471  867854563 1093350727  876790370 1611517182 1373612644
[43] 1981591207 1909864167 1000465243 1777613215  851424758 1312213323
[49] 1310570899 2267765132 2289238173 1184476195  871394275 1415568281
[55]  690825057 1606646358 2122773538 1444877241  986584048  984152394
[61] 1349778437 1191123871 2808204983 1323805141 3003228123 3051638715
[67] 1630505498 2154517901 1772055161 2033547791 2749314815 3154533992
[73] 2535079543 1055800681 1804861494 1331263664  758229742 1611667454
[79] 1010275270 1156148220         NA         NA         NA         NA
[85]         NA         NA         NA         NA         NA         NA
[91]         NA         NA         NA         NA         NA         NA

# bowtie2 index T2T genome

# alignment

In [654]:
species = 't2t'
if(species == 'hg38') {
    ref_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/YuannyuZhang/Data/UCSC/hg38/Bowtie2Index'
}else if(species == 'hg19'){
    ref_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/YuannyuZhang/Data/For_RNA-Seq/Homo_sapiens/UCSC/hg19/Sequence/Bowtie2Index'
}else if(species == 'mosue'){
    ref_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/YuannyuZhang/Data/For_RNA-Seq/Mus_musculus/UCSC/mm10/Sequence/Bowtie2Index'
}else if(species == 't2t'){
    ref_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/genomes/t2t/bowtie2_index/bowtie2_index'
}

In [ ]:
$ bowtie2 -p 2 -q --local \
-x /n/groups/shared_databases/bowtie2_indexes/mm10 \
-U ~/chipseq_workshop/raw_data/wt_sample2_chip.fastq.gz \
-S ~/chipseq_workshop/results/bowtie2/wt_sample2_chip.sam


In [674]:
# module load bow2/2.7.10a
dsn[, fname_pre := paste0(sample_dir, sample_name, '_', species)]

dsn[, bow2_jobname := paste0('bow2_', sample_name)]
dsn[, bow2_oo := paste0(sample_dir, 'bow2_', sample_name, '_out')]
dsn[, bow2_eo := paste0(sample_dir, 'bow2_', sample_name, '_err')]
dsn[, sam_file := paste0(fname_pre, '.sam')]
dsn[, sam_report_file := paste0(fname_pre, '_bowtie2Report.txt')]

# -k aln times
dsn[, bow2_cmd := paste0('bsub -J ', bow2_jobname, ' -o ', bow2_oo, ' -e ', bow2_eo, ' -P ', PP, ' -n 30 -R "rusage[mem=5GB]" ')]
dsn[, bow2_cmd := paste0(bow2_cmd, ' -W 90')]
dsn[file.exists(RR2),  bow2_cmd := paste0(bow2_cmd, ' " bowtie2 -k 1 -x ', ref_dir, ' -1 ', fastq_r1_trimmed, ' -2 ', fastq_r2_trimmed)]
dsn[!file.exists(RR2), bow2_cmd := paste0(bow2_cmd, ' " bowtie2 -k 1 -x ', ref_dir, ' -U ', fastq_r1_trimmed, ' ')]
dsn[, bow2_cmd := paste0(bow2_cmd, ' -S ', sam_file, ' "')]

write(dsn[81:96, bow2_cmd], file = paste0(wk_dir, 'run_', PP, '_bow2.sh'))

# 81 -- 96 is single end, not paired

In [677]:
head( dsn[81:96, .(sample_name_manual, RR1_sample, RR2_sample)] )

sample_name_manual,RR1_sample,RR2_sample
<chr>,<chr>,<chr>
D1_HSPC_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D1_HSPC_1/D1_HSPC_1_R1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D1_HSPC_1/D1_HSPC_1_R2.fastq.gz
D2_HSPC_2,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D2_HSPC_2/D2_HSPC_2_R1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D2_HSPC_2/D2_HSPC_2_R2.fastq.gz
D3_BFUE_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D3_BFUE_1/D3_BFUE_1_R1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D3_BFUE_1/D3_BFUE_1_R2.fastq.gz
D4_BFUE_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4_BFUE_1/D4_BFUE_1_R1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4_BFUE_1/D4_BFUE_1_R2.fastq.gz
D5_CFUE_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D5_CFUE_1/D5_CFUE_1_R1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D5_CFUE_1/D5_CFUE_1_R2.fastq.gz
D6_CFUE_1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D6_CFUE_1/D6_CFUE_1_R1.fastq.gz,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D6_CFUE_1/D6_CFUE_1_R2.fastq.gz


In [689]:
dsn[batch == '', .(sample_name_manual, srr_id, batch)]

sample_name_manual,srr_id,batch
<chr>,<chr>,<chr>
D1_Neu_1,SRR2999324,
D2_Neu_1,SRR2999325,
D1_DC_1,SRR5210376,
D1003_Mono_1,SRR7650849,
D1_preproB_1,SRR8247848,
D2_preproB_2,SRR8247849,
D3_preproB_3,SRR8247850,
ICH13195_ProB_1,SRR8247851,
ICH13475_ProB_1,SRR8247852,


In [678]:
table(dsn$batch)


          gse128266  gse74912 
       10        16        70 

In [690]:
table(dsn[batch == 'gse128266', cell_type])
table(dsn[batch == 'gse74912', cell_type])


  BFUE   CFUE  Ebaso   HSPC orthoE  polyE   proE 
     2      2      4      2      2      2      2 


     B  blast CBCD34   CD34    CD4    CD8    CLP    CMP     EB    GMP    HSC 
     3     15      1      2      4      4      3      1      3      1      4 
   LSC    MEP    MPP     NK   pHSC 
     8      1      1      4     15 

In [663]:
dim(dsn)

[1] 96 55

In [228]:
file.exists(dsn$sam_file)
file.size(dsn$sam_file)

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[25] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[37] FALSE FALSE

# samtools: sam to bam

In [691]:
dsn[, sam2bam_jobname := paste0('sam2bam_', sample_name)]
dsn[, sam2bam_oo := paste0(sample_dir, 'sam2bam_', sample_name, '_out')]
dsn[, sam2bam_eo := paste0(sample_dir, 'sam2bam_', sample_name, '_err')]
dsn[, bam_file := paste0(fname_pre, '.bam')]

# -k aln times
dsn[, sam2bam_cmd := paste0('bsub -J ', sam2bam_jobname, ' -o ', sam2bam_oo, ' -e ', sam2bam_eo, ' -P ', PP, ' -n 30 -R "rusage[mem=5GB]" ')]
dsn[, sam2bam_cmd := paste0(sam2bam_cmd, ' -W 90')]
dsn[, sam2bam_cmd := paste0(sam2bam_cmd, ' " samtools view -bS ', sam_file, ' > ', bam_file, '"')]

write(dsn[, sam2bam_cmd], file = paste0(wk_dir, 'run_', PP, '_sam2bam.sh'))


In [668]:
dsn[is.na(sam_file), RR1_sample]
#file.exists(dsn[is.na(sam_file), RR1_sample])

character(0)

In [695]:
file.size(dsn$bam_file)
file.size(dsn$sam_file)

[1]  798408706 1059583573 1520442660 1653431143 3690832502 3278247793
 [7] 4269193351 2070455997 1983002232 7980637103 7128168743 3519173192
[13] 1831549259 7946176077 3005024650 2953905037 3743518427 6686223089
[19] 7791550974 5772752471 3854954272 6318686403 6683019287 6776287363
[25] 7834811948 7574288958 2374334115 1281433910 4877049169 1985497309
[31] 1502548497 4846583663  852723131 4627274553 3778956035 2625766370
[37] 2260146899 2163585771 2888111876 2114898261 4253684528 3513190294
[43] 5170894555 4822735008 2524967688 4149892019 2245858268 3466589138
[49] 3166143336 5138610617 6017606642 3008784872 2320044381 3605881353
[55] 1806174155 4256140594 5080699063 3717366472 2479122798 2573974897
[61] 3520436166 3091548458 6688632341 3443481105 6922593635 7082517865
[67] 4227485480 5469567129 4563474973 5047689805 4830898649 5266062982
[73] 6167868262 2700340497 5822262137 4272135717 2394493440 5202246612
[79] 3265681368 3741708867 2954459476 2972146265 2520902055 3803686774
[85] 5373953893 1694245372 2058211432 3759990177 2296874778 3446083612
[91] 2382408470 3570891836 2095863069 3764667423 2017364575 3875589462

[1]  3485434995  4071114418  6416358700  6909465317 12183849813 11923544054
 [7] 15497770292  7442169916 11876371392 27077607424 25509265408 11043982734
[13] 10345284967 24995971072  9326796684  9125828963 11647023245 20795454498
[19] 28077694976 22842916864 15322770676 25258278912 26798424064 24356528128
[25] 28104884224 27274081408  7509259186  4063823839 17774228784  6844435437
[31]  5201385925 18508833106  2790799565 19134981157 14800369857 10032714181
[37]  8721066840  9067436786 11225191935  8986476571 16637594171 13765902082
[43] 20232126476 19029792070  9591827301 16223930861  9229186999 13660972074
[49] 13546359868 22663837367 23334939726 12469974088  8783141526 13835816016
[55]  6833731084 16297038883 19959967028 15345679148 10283009644 10241131450
[61] 13856819863 12358893335 27033993216 13499874044 27499872256 25686212608
[67] 16967984869 21673359166 18022764952 19980426709 20454121472 21894070272
[73] 30921515008 12479080385 27719951853 20262301432 11422035706 24859606484
[79] 15565222213 17934127813 10305508971 10344112065  8804125866 14949870341
[85] 21309305012  5527936656  7345357192 12121570211  8167360460 11147881067
[91]  8336808504 11499402152  7310491121 12203163666  7026400855 12355606590

# bedtools: bam2bed

In [697]:
dsn[, bam2bed_jobname := paste0('bam2bed_', sample_name)]
dsn[, bam2bed_oo := paste0(sample_dir, 'bam2bed_', sample_name, '_out')]
dsn[, bam2bed_eo := paste0(sample_dir, 'bam2bed_', sample_name, '_err')]
dsn[, bed_file := paste0(fname_pre, '.bed')]

dsn[, bam2bed_cmd := paste0('bsub -J ', bam2bed_jobname, ' -o ', bam2bed_oo, ' -e ', bam2bed_eo, ' -P ', PP, ' -n 8 -R "rusage[mem=5GB]" ')]
dsn[, bam2bed_cmd := paste0(bam2bed_cmd, ' -W 90')]
dsn[, bam2bed_cmd := paste0(bam2bed_cmd, ' " bedtools bamtobed -i ', bam_file, ' > ', bed_file, '"')]

write(dsn[, bam2bed_cmd], file = paste0(wk_dir, 'run_', PP, '_bam2bed.sh'))


In [702]:
head(dsn$bed_file)

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/D4983_HSC_1_t2t.bed"      
[2] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D6792_HSC_1/D6792_HSC_1_t2t.bed"      
[3] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D7291_CD34_1/D7291_CD34_1_t2t.bed"    
[4] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D7292_CD34_2/D7292_CD34_2_t2t.bed"    
[5] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D1020_CBCD34_1/D1020_CBCD34_1_t2t.bed"
[6] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D1022_B_1/D1022_B_1_t2t.bed"

# bed2normed_wig

In [734]:
# module load bed2wig/2.7.10a
chromSize_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/YuannyuZhang/Data/chromSize'
python_dir    = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/YuannyuZhang/scripts/python_scripts/'
chrom_size_file = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/genomes/t2t/t2t_sizes_genome'

dsn[, wig_file := paste0(fname_pre, '.wig') ]

# calculate norm factor
# it takes time, comment out saving time
# these steps are requested for new calculation
#dsn[, fac := {system(paste0('wc -l ', bed_file, ' | cut -f 1 -d " "'), intern = T)}, by = 1:nrow(dsn)]
#dsn[, fac := as.numeric(fac) ]
dsn[, fac_n := 10000000 / fac] 
dsn[, fac_nn := paste0(fac_n, ',0')]

dsn[, bed2wig_jobname := paste0('bed2wig_', sample_name)]
dsn[, bed2wig_oo := paste0(sample_dir, 'bed2wig_', sample_name, '_out')]
dsn[, bed2wig_eo := paste0(sample_dir, 'bed2wig_', sample_name, '_err')]

dsn[, bed2wig_cmd := paste0('bsub -J ', bed2wig_jobname, ' -o ', bed2wig_oo, ' -e ', bed2wig_eo, ' -P ', PP, ' -n 3 -R "rusage[mem=5GB]" ')]
dsn[, bed2wig_cmd := paste0(bed2wig_cmd, ' -W 90')]
dsn[, bed2wig_cmd := paste0(bed2wig_cmd, ' " python3 ', '/home/whu78/script/iBed2Wiggle.py --inFile ', bed_file, ' --outFile ', sample_dir, ' ')] 
dsn[, bed2wig_cmd := paste0(bed2wig_cmd, ' --chromSize ', chrom_size_file, ' --normFactor ', fac_nn, ' "')] 

write(dsn[, bed2wig_cmd], file = paste0(wk_dir, 'run_', PP, '_bed2wig_step1.sh'))


In [729]:
head(dsn$sample_dir)

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/"   
[2] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D6792_HSC_1/"   
[3] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D7291_CD34_1/"  
[4] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D7292_CD34_2/"  
[5] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D1020_CBCD34_1/"
[6] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D1022_B_1/"

In [736]:
file.size(dsn$wig_file)


[1] 1189183225 1356556251 1878710308 1987268752 2662924423 3097129061
 [7] 1500639760 1887896487 1853552074 2589163874 2856699752 3160081836
[13] 1833297268 2910301171 2796379554 1467947775 2341833377 2538489825
[19] 2501667033 4005092204 3367165382 3352817887 3583411606 2923045870
[25] 2086266235 1546023955 1722223895  785350462 1985467335 1291146258
[31] 1216387771 2365211053 1041624339 1971521584 3305494408 2773806037
[37] 2743192331 2624330549 2666519518 2915246736 2962296668 2061044962
[43] 2909932857 2199765624 2447554119 2555503222 2456020680 3085795906
[49] 2416104988 2375051374 3224827559 1796658390 2455487194 2092156818
[55] 2072542289 3179286704 2733009683 2561346185 1707897200 2310895851
[61] 3040478446 1983323971 3924247685 3347792949 3025640459 3986300383
[67] 2181591061 2942387457 2615310184 2408535372 4090574008 4056710019
[73]         NA 3765476108         NA 1813915768 2343769212         NA
[79] 1438238557  376915509 2285610045 2528995209 2425408130 3014070818
[85]         NA 2816961584 1347255934 2219886978  929136023 2228490797
[91] 1293232059 2228173627 1293183683 2777042925  971377768 2199614165

In [725]:
dsn[1, .(bed_file, fac, fac_n)]

bed_file,fac,fac_n
<chr>,<dbl>,<dbl>
/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/D4983_HSC_1_t2t.bed,10140537,0.9861411


In [237]:
# rsync -aP rsync://hgdownload-sd.soe.ucsc.edu/genome/admin/exe/linux.x86_64/ /home/whu78/repos/ucsc_tools/wigToBigWig


In [742]:
# step2
# module load bed2wig2/2.7.10a
dsn[, bigwig_file := paste0(fname_pre, '.bw') ]

dsn[, bed2wig2_jobname := paste0('bed2wig2_', sample_name)]
dsn[, bed2wig2_oo := paste0(sample_dir, 'bed2wig2_', sample_name, '_out')]
dsn[, bed2wig2_eo := paste0(sample_dir, 'bed2wig2_', sample_name, '_err')]

dsn[, bed2wig2_cmd := paste0('bsub -J ', bed2wig2_jobname, ' -o ', bed2wig2_oo, ' -e ', bed2wig2_eo, ' -P ', PP, ' -n 3 -R "rusage[mem=5GB]" ')]
dsn[, bed2wig2_cmd := paste0(bed2wig2_cmd, ' -W 90')]
dsn[, bed2wig2_cmd := paste0(bed2wig2_cmd, ' " wigToBigWig -clip ', wig_file, ' ', chrom_size_file, ' ', bigwig_file, ' " ')]

write(dsn[, bed2wig2_cmd], file = paste0(wk_dir, 'run_', PP, '_bed2wig_step2.sh'))


In [744]:
dsn[1, bed2wig2_cmd]

[1] "bsub -J bed2wig2_D4983_HSC_1 -o /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/bed2wig2_D4983_HSC_1_out -e /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/bed2wig2_D4983_HSC_1_err -P atac -n 3 -R \"rusage[mem=5GB]\"  -W 90 \" wigToBigWig -clip /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/D4983_HSC_1_t2t.wig /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/genomes/t2t/t2t_sizes_genome /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/D4983_HSC_1_t2t.bw \" "

In [749]:
file.size(dsn$bigwig_file)

[1] 147875602 169517472 197971159 241167179 322394319 276701184 190572303
 [8] 226294639 239547529 333816551 370180096 212918272 232318240 365207552
[15] 301211648 164768441 285783922 326324045 325019078 188284928 240508928
[22] 250380288 214695936 318455808 284192503 213154938 178825058  89813495
[29] 253810244 140636407 131245841 307403487 112508683 257327054 211320832
[36] 322887680 328317284 319882380 327052089 281829376 299843584 262199520
[43] 197263360 285943410 299335281 313815010 297680710 229138432 300838851
[50] 297263488 364675072 231546846 297739419 262646213 252873537 221462528
[57] 298164224 318929066 224948570 287715244 231194624 252541388 163307520
[64] 320765952 292413440 163938304 290837915 275529728 330650746 306630580
[71] 322134016 138289152        NA 191660032 323809608 251183873 307648224
[78] 298522258 202611910 329408512 217414540 277635981 230654734 177684480
[85] 201236480 147898368 130981310 210556065  93605734 207998567 126918680
[92] 209088954 126686958 236879872  97687384 209920178

# Qc

In [ ]:
library(ATACseqQC)

# peak calling 

In [739]:
# step2
# module load macs2/2.7.10a

qvalue_cutoff = 0.05 

dsn[, macs2_jobname := paste0('macs2_', sample_name)]
dsn[, macs2_oo := paste0(sample_dir, 'macs2_', sample_name, '_out')]
dsn[, macs2_eo := paste0(sample_dir, 'macs2_', sample_name, '_err')]

dsn[, macs2_cmd := paste0('bsub -J ', macs2_jobname, ' -o ', macs2_oo, ' -e ', macs2_eo, ' -P ', PP, ' -n 3 -R "rusage[mem=5GB]" ')]
dsn[, macs2_cmd := paste0(macs2_cmd, ' -W 90')]
dsn[, macs2_cmd := paste0(macs2_cmd, ' " macs2 callpeak -q ', qvalue_cutoff, ' -t ', bed_file, ' --name ', fname_pre, '_bowtie2_nomodel --nomodel -f BED "')]

write(dsn[, macs2_cmd], file = paste0(wk_dir, 'run_', PP, '_macs.sh'))


In [740]:
dsn[1, macs2_cmd]

[1] "bsub -J macs2_D4983_HSC_1 -o /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/macs2_D4983_HSC_1_out -e /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/macs2_D4983_HSC_1_err -P atac -n 3 -R \"rusage[mem=5GB]\"  -W 90 \" macs2 callpeak -q 0.05 -t /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/D4983_HSC_1_t2t.bed --name /research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/D4983_HSC_1_t2t_bowtie2_nomodel --nomodel -f BED \""

In [750]:
file.size(dsn$bed_file)
file.size(dsn$bigwig_file)

[1]  511224706  623747553  596084389  720630412 1291562627 1101402603
 [7] 1223635944  764912169 1045941592 2886878461 3261119809 1157996092
[13]  853485673 2262673883  925819257  851908929 1061326671 1939006228
[19] 2702910767 3140465166 2081605904 2935553003 3051504342 2407774925
[25] 2504108865 2198713812  525871112  227279256 1583149989  507846577
[31]  373967692 2477089250  360008471 1883414337 3106711748 1776013719
[37] 1584816409 1043950202 1541695804  899296162 2423518339 1777988039
[43] 2835270881 2317311365 1136121060 1267594034 1339503886 1987024381
[49] 1395579025 1554478161 3113391476 1614000233 1280825958 1708584802
[55]  915247881 2220125591 1825382943 2038380749 1233396475 1437217270
[61] 1897608806 1714978832 2999467950 1877112486 3638025774 3105771041
[67] 2324835464 2681361438 2426961849 2286513511 3133652796 3371176717
[73] 7418619812 2410280107 6745574661 4859782443 2595763883 5921356880
[79] 3727190857 4307356329 1029543061 1093264593 1304488397 2277384150
[85] 2999920276  898132370  847473741 1485879839  868277197 1386705298
[91]  801862981 1407968449  710240188 1776285026  548783770 1676000511

[1] 147875602 169517472 197971159 241167179 322394319 365772721 190572303
 [8] 226294639 239547529 333816551 372557006 374753894 232318240 375833467
[15] 329124744 164768441 285783922 326324045 325019078 506750391 411702801
[22] 415713744 450015445 373517966 284192503 213154938 178825058  89813495
[29] 253810244 140636407 131245841 307403487 112508683 257327054 416561676
[36] 338771932 328317284 319882380 327052089 338726209 372750447 262199520
[43] 366800126 285943410 299335281 313815010 297680710 378219714 300838851
[50] 297263488 408744593 231546846 297739419 262646213 252873537 399134758
[57] 348021266 318929066 224948570 287715244 371005404 252541388 499154596
[64] 403903477 398035531 507495128 290837915 372052605 330650746 306630580
[71] 509924563 513819998        NA 466565639 323809608 251183873 307648224
[78] 298522258 202611910 414398209 217414540 277635981 230654734 341666601
[85] 340093648 299883531 130981310 210556065  93605734 207998567 126918680
[92] 209088954 126686958 308019767  97687384 209920178

In [9]:
dsn$sample_dir[1]

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/run_atac_merged/D4983_HSC_1/"

In [11]:
dsn[, peak_file := paste0(sample_dir, sample_name, '_t2t_bowtie2_nomodel_peaks.narrowPeak')] 
file.exists(dsn$peak_file)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[91] TRUE TRUE TRUE TRUE TRUE TRUE

In [ ]:
D4983_HSC_1_t2t_bowtie2_nomodel_peaks.narrowPeak

# export design table 

In [18]:
fwrite(dsn, file = 'data/dsn_atac_12Jul.tsv', sep = '\t')

In [11]:
dsn = fread('data/dsn_atac_12Jul.tsv')

In [12]:
cell_types = c('HSC', 'HSPC', 'MPP', 'CMP', 'GMP', 'MEP', 'CLP', 'CBCD34', 'LSC', 'pHSC', 'CD34', 'blast', 'preproB', 'ProB', 'B', 'DC', 
               'CD4', 'CD8', 'NK', 'Mono', 'Neu', 'EB', 'BFUE', 'CFUE', 'proE', 'Ebaso', 'polyE', 'orthoE')
               
dsn[, cell_type := factor(cell_type, levels = cell_types)]

In [13]:
table(dsn$cell_type)


    HSC    HSPC     MPP     CMP     GMP     MEP     CLP  CBCD34     LSC    pHSC 
      4       2       1       1       1       1       3       1       8      15 
   CD34   blast preproB    ProB       B      DC     CD4     CD8      NK    Mono 
      2      15       3       3       3       1       4       4       4       1 
    Neu      EB    BFUE    CFUE    proE   Ebaso   polyE  orthoE 
      2       3       2       2       2       4       2       2 

In [14]:
getwd()

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan"

In [22]:
save.image(file = 'nb_TE_HSC_ATAC.rdata')

In [15]:
load('nb_TE_HSC_ATAC.rdata')

In [8]:
file.size('nb_TE_HSC_ATAC.rdata')

[1] 273206

# delete sam file

- do it in the terminal
- trimmed fastq file
- merged fastq file
- sam file
- are all removed 